### Entrenamiento, Predicción y Evaluación de un modelo MLP

#### **Entrenamiento**

---

#### **Predicción**

A un paso

A múltiples pasos

---

#### **Analisis de resultados**

Métricas a evaluar: 
1. sMAPE
2. WAPE

---